# DB Mol BI
ETL para extração dos dados necessários para a análise de BI do DB Mol

In [45]:
import pandas as pd
import re

## output sem cortes
# pd.set_option('display.max_rows', None)

In [46]:
## ler arquivo dbmol_completo.tsv na pasta dados, ler 500 linhas
dbmol = pd.read_csv('dados/dbmol_completo.tsv', sep='\t') #, nrows=500)
# dbmol = pd.read_csv('dados/dbmol_completo.tsv', sep='\t')


In [47]:
dbmol.head(20)

,Cidade,DataCadastro,DataLiberacaoClinica,CodigoProcedimento,DataNascimento,NumeroPedido,Parametro,Procedimento,Resultado,Sexo,UF
0,ALVARES MACHADO,2023-09-23 00:00:00,2023-09-26 00:00:00,AHCV,1973-05-17 00:00:00,1243132120,AHCV,HEPATITE C - ANTI - HCV,0.11,F,SP
1,ALVARES MACHADO,2023-09-23 00:00:00,2023-09-26 00:00:00,AHCV,1973-05-17 00:00:00,1243132120,METODO,HEPATITE C - ANTI - HCV,Método..: QUIMIOLUMINESCÊNCIA,F,SP
2,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1944-03-16 00:00:00,1243177974,AHBE,HEPATITE B - ANTI - HBE,1.44,F,SP
3,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1944-03-16 00:00:00,1243177974,METODO,HEPATITE B - ANTI - HBE,Método..: QUIMIOLUMINESCÊNCIA,F,SP
4,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1966-07-19 00:00:00,1243173758,AHBE,HEPATITE B - ANTI - HBE,1.45,M,SP
5,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1966-07-19 00:00:00,1243173758,METODO,HEPATITE B - ANTI - HBE,Método..: QUIMIOLUMINESCÊNCIA,M,SP
6,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1984-11-14 00:00:00,1243177936,NaN,HEPATITE B - ANTI - HBE,Resultado conferido e liberado.,M,SP
7,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1984-11-14 00:00:00,1243177936,AHBE,HEPATITE B - ANTI - HBE,Inferior a 0.20,M,SP
8,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1984-11-14 00:00:00,1243177936,METODO,HEPATITE B - ANTI - HBE,Método..: QUIMIOLUMINESCÊNCIA,M,SP
9,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,CMVG,1982-01-05 00:00:00,1243177817,CMVG,CITOMEGALOVÍRUS - ANTICORPOS IGG (CLIA),Superior a 250.00,F,SP


In [48]:
# print max da coluna resultado
print(dbmol_filtrado['Resultado'].max())

100000.0


In [49]:
# Filtrar linhas onde 'Parametro' não é 'METODO'
dbmol_filtrado = dbmol[dbmol['Parametro'] != 'METODO']

## funçao para extrair valores int e float da coluna 'Resultado'
def extrair_valores(texto):
    # Encontrar todos os números inteiros e decimais em 'Resultado'
    valores = re.findall(r'[-+]?\d*\.\d+|\d+', str(texto))
    # Se a lista 'valores' não estiver vazia, converter o primeiro valor para float
    if valores:
        return float(valores[0])
    # Se a lista 'valores' estiver vazia, retornar None
    else:
        return None

# ## aplicar função extrair_valore
dbmol_filtrado['Resultado'] = dbmol_filtrado['Resultado'].apply(extrair_valores)

# Encontrar o valor mínimo e máximo da coluna 'Resultado'
resultado_min = dbmol_filtrado['Resultado'].min()
resultado_max = dbmol_filtrado['Resultado'].max()

## arredondar valores das colunas min e max para duas casas apos a virgula se float
# resultado_min = round(resultado_min, 2)
# resultado_max = round(resultado_max, 2)

## adicionar estas colunas ao dataset, para cada valor unico de procedimento
dbmol_filtrado['Resultado_min'] = resultado_min
dbmol_filtrado['Resultado_max'] = resultado_max

# # ## criar colunas baseado em Resultado
dbmol_filtrado.head()


/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_22264/443948035.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbmol_filtrado['Resultado'] = dbmol_filtrado['Resultado'].apply(extrair_valores)
/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_22264/443948035.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dbmol_filtrado['Resultado_min'] = resultado_min
/var/folders/gt/c4rqf02d5m731j41w9p5mt2h0000gp/T/ipykernel_22264/443948035.py:28: SettingWithCopyWarning: 
A value is trying to

,Cidade,DataCadastro,DataLiberacaoClinica,CodigoProcedimento,DataNascimento,NumeroPedido,Parametro,Procedimento,Resultado,Sexo,UF,Resultado_min,Resultado_max
0,ALVARES MACHADO,2023-09-23 00:00:00,2023-09-26 00:00:00,AHCV,1973-05-17 00:00:00,1243132120,AHCV,HEPATITE C - ANTI - HCV,0.11,F,SP,-0.01,2.300212e+11
2,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1944-03-16 00:00:00,1243177974,AHBE,HEPATITE B - ANTI - HBE,1.44,F,SP,-0.01,2.300212e+11
4,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1966-07-19 00:00:00,1243173758,AHBE,HEPATITE B - ANTI - HBE,1.45,M,SP,-0.01,2.300212e+11
6,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1984-11-14 00:00:00,1243177936,NaN,HEPATITE B - ANTI - HBE,NaN,M,SP,-0.01,2.300212e+11
7,AMERICANA,2023-09-23 00:00:00,2023-09-26 00:00:00,AHBE,1984-11-14 00:00:00,1243177936,AHBE,HEPATITE B - ANTI - HBE,0.20,M,SP,-0.01,2.300212e+11


In [50]:
# Função para verificar se a string contém um número
def has_number(input_string):
    return bool(re.search(r'\d', str(input_string)))

# Função para verificar se a string é um número negativo, contém 'negativo' ou 'Inferior'
def is_min(value):
    if str(value).startswith('-') or 'negativo' in str(value) or 'Inferior' in str(value):
        return True
    if has_number(str(value)):
        number = re.findall(r"[-+]?\d*\.\d+|\d+", str(value))
        if number and float(number[0]) < 0:
            return True
    return False

# Função para verificar se a string é um número positivo, contém 'positivo' ou 'superior'
def is_max(value):
    if str(value).startswith('+') or 'positivo' in str(value) or 'Superior' in str(value):
        return True
    if has_number(str(value)):
        number = re.findall(r"[-+]?\d*\.\d+|\d+", str(value))
        if number and float(number[0]) > 0:
            return True
    return False

# Aplicando as funções para criar novas colunas 'Min' e 'Max'
dbmol['Min'] = dbmol['Resultado'].apply(is_min)
dbmol['Max'] = dbmol['Resultado'].apply(is_max)
dbmol['HasNumber'] = dbmol['Resultado'].apply(has_number)

# Filtrando as linhas onde 'Min' ou 'Max' é True
dbmol_min = dbmol[dbmol['Min']]
dbmol_max = dbmol[dbmol['Max']]
dbmol_has_number = dbmol[dbmol['HasNumber']]

In [51]:
dbmol.columns

Index(['Cidade', 'DataCadastro', 'DataLiberacaoClinica', 'CodigoProcedimento',
       'DataNascimento', 'NumeroPedido', 'Parametro', 'Procedimento',
       'Resultado', 'Sexo', 'UF', 'Min', 'Max', 'HasNumber'],
      dtype='object')

In [52]:
dbmol['Procedimento'].unique()

array(['HEPATITE C - ANTI - HCV', 'HEPATITE B - ANTI - HBE',
       'CITOMEGALOVÍRUS - ANTICORPOS IGG (CLIA)',
       'CITOMEGALOVÍRUS - ANTICORPOS IGM (CLIA)',
       'FTA - ABS - ANTICORPOS IGG', 'HEPATITE B - HBEAG',
       'UROCULTURA E ANTIBIOGRAMA COM CONTAGEM DE COLÔNIAS',
       'CULTURA E ANTIBIOGRAMA - LAVADO BRONCO-ALVEOLAR',
       'RUBÉOLA - ANTICORPOS IGG (CLIA)', 'HEPATITE B - ANTI-HBC TOTAL',
       'HEPATITE B - ANTI - HBC - IGM', 'HEPATITE B - ANTI - HBS',
       'RUBÉOLA - ANTICORPOS IGM (CLIA)', 'HEPATITE B - HBSAG',
       'DENGUE - ANTICORPOS IGG', 'DENGUE - ANTICORPOS IGM',
       'CITOMEGALOVÍRUS AVIDEZ - ANTICORPOS IGG (CLIA)',
       'UROCULTURA E ANTIBIOGRAMA - ÁCIDO BÓRICO',
       'CULTURA E ANTIBIOGRAMA - SECREÇÃO VAGINAL',
       'CULTURA E ANTIFUNGIGRAMA - FUNGOS',
       'HERPES 1 E 2 - ANTICORPOS IGM', 'PARVOVÍRUS - ANTICORPOS IGG',
       'HEPATITE C - DETECÇÃO POR PCR',
       'HEPATITE DELTA VÍRUS ANTICORPOS (ANTI-HDV)',
       'HIV - DETECÇÃO POR P

In [53]:
# ## fazer um dataframe  apontando os valores da coluna Procedimento e o min e max da coluna Resultado, sem repetir valores
# dbmol_itps = dbmol[dbmol['HasNumber']].groupby('Procedimento')['Resultado'].agg(['min', 'max'])
# dbmol_itps
# # dbmol_itps = dbmol[dbmol['Resultado'].str.contains('\d', na=False)].groupby('Procedimento')['Resultado'].agg(['min', 'max'])

# dbmol_itps

## Filtros por patógenos de interesse prioritário para o ITpS

In [54]:
## filtro para a mesma coluna, considerando as strings FEBRE AMARELA, DENGUE, ZIKA e CHIKUNGUNYA, OROPOUCHE, SARS-COV-2, COVID-19, FLU A, FLU B, INFLUENZA A, INFLUENZA B, VIRUS SINCICIAL RESPIRATORIO
dbmol_pat = dbmol[dbmol['Procedimento'].str.contains('FEBRE AMARELA|DENGUE|ZIKA|CHIKUNGUNYA|OROPUCHE|SARS-COV-2|COVID-19|COVID|SARS|FLU A|FLU B|INFLUENZA A|INFLUENZA B|VIRUS SINCICIAL RESPIRATORIO|PAINEL')]

In [55]:
## selecionar valores que possuam as palavras "ANTIGENO", "PCR" e "ANTICORPOS" na coluna Procedimento
dbmol_check = dbmol_pat[dbmol_pat['Procedimento'].str.contains('ANTIGENO|PCR|ANTICORPOS|IGG|IGM|IGA')]

In [56]:
dbmol_pat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106106 entries, 391 to 25688427
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Cidade                106106 non-null  object
 1   DataCadastro          106106 non-null  object
 2   DataLiberacaoClinica  106078 non-null  object
 3   CodigoProcedimento    106106 non-null  object
 4   DataNascimento        106106 non-null  object
 5   NumeroPedido          106106 non-null  int64 
 6   Parametro             97940 non-null   object
 7   Procedimento          106106 non-null  object
 8   Resultado             106106 non-null  object
 9   Sexo                  106106 non-null  object
 10  UF                    106106 non-null  object
 11  Min                   106106 non-null  bool  
 12  Max                   106106 non-null  bool  
 13  HasNumber             106106 non-null  bool  
dtypes: bool(3), int64(1), object(10)
memory usage: 10.0+ MB


In [57]:
dbmol_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90815 entries, 391 to 25688427
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Cidade                90815 non-null  object
 1   DataCadastro          90815 non-null  object
 2   DataLiberacaoClinica  90813 non-null  object
 3   CodigoProcedimento    90815 non-null  object
 4   DataNascimento        90815 non-null  object
 5   NumeroPedido          90815 non-null  int64 
 6   Parametro             82667 non-null  object
 7   Procedimento          90815 non-null  object
 8   Resultado             90815 non-null  object
 9   Sexo                  90815 non-null  object
 10  UF                    90815 non-null  object
 11  Min                   90815 non-null  bool  
 12  Max                   90815 non-null  bool  
 13  HasNumber             90815 non-null  bool  
dtypes: bool(3), int64(1), object(10)
memory usage: 8.6+ MB


In [58]:
dbmol_pat['Procedimento'].unique()

array(['DENGUE - ANTICORPOS IGG', 'DENGUE - ANTICORPOS IGM',
       'ZIKA VÍRUS ANTICORPOS IGG', 'ZIKA VÍRUS ANTICORPOS IGM',
       'PAINEL DE VÍRUS RESPIRATÓRIO SARS-COV-2, VÍRUS SINCICIAL, INFLUENZA A, INFLUENZA B',
       'DENGUE NS1', 'ANTICORPOS ANTI CHIKUNGUNYA IGG E IGM',
       'INFLUENZA A VÍRUS ANTICORPOS IGM',
       'INFLUENZA A VÍRUS ANTICORPOS IGG',
       'CORONAVÍRUS 2019 - SARS-COV-2 IGG QUANTITATIVO',
       'TESTE DE NEUTRALIZAÇÃO SARS-COV-2/COVID19, ANTICORPOS TOTAIS - SORO',
       'CORONAVÍRUS 2019 ANTICORPOS IGA (COVID19)',
       'PAINEL DE ARBOVIROSES (DENGUE, ZIKA E CHIKUNGUNYA)',
       'CHIKUNGUNYA VÍRUS IGG', 'CHIKUNGUNYA VÍRUS IGM',
       'PAINEL MOLECULAR PARA INFECÇÕES NEUROLÓGICAS (LÍQUOR)',
       'INFLUENZA B VÍRUS ANTICORPOS IGG',
       'INFLUENZA B VÍRUS ANTICORPOS IGM',
       'ZIKA VÍRUS DETECÇÃO POR PCR - PLASMA',
       'INFLUENZA A E B - DETECÇÃO POR PCR',
       'PESQUISA MOLECULAR DO VÍRUS CHIKUNGUNYA',
       'DETECÇÃO E TIPAGEM DO VÍRUS 

In [59]:
## listar os valores únicos da coluna Procedimento
dbmol_check['Procedimento'].unique()

array(['DENGUE - ANTICORPOS IGG', 'DENGUE - ANTICORPOS IGM',
       'ZIKA VÍRUS ANTICORPOS IGG', 'ZIKA VÍRUS ANTICORPOS IGM',
       'ANTICORPOS ANTI CHIKUNGUNYA IGG E IGM',
       'INFLUENZA A VÍRUS ANTICORPOS IGM',
       'INFLUENZA A VÍRUS ANTICORPOS IGG',
       'CORONAVÍRUS 2019 - SARS-COV-2 IGG QUANTITATIVO',
       'TESTE DE NEUTRALIZAÇÃO SARS-COV-2/COVID19, ANTICORPOS TOTAIS - SORO',
       'CORONAVÍRUS 2019 ANTICORPOS IGA (COVID19)',
       'CHIKUNGUNYA VÍRUS IGG', 'CHIKUNGUNYA VÍRUS IGM',
       'INFLUENZA B VÍRUS ANTICORPOS IGG',
       'INFLUENZA B VÍRUS ANTICORPOS IGM',
       'ZIKA VÍRUS DETECÇÃO POR PCR - PLASMA',
       'INFLUENZA A E B - DETECÇÃO POR PCR',
       'ESTUDO SOROLÓGICO -  VÍRUS FEBRE AMARELA IGG E IGM',
       'FEBRE AMARELA - DETECÇÃO POR PCR'], dtype=object)

In [60]:
## mostrar valores únicos da coluna Procedimento e Resultado, excluindo strings que tenham números
dbmol_pat[~dbmol_pat['Resultado'].str.contains('\d')]['Resultado'].unique()

array(['Método..: ENZIMAIMUNOENSAIO', 'Não detectado',
       'Swab Nasofaringe e Orofaringe', 'Método..: RT-PCR', 'Negativo',
       'Método..: IMUNOCROMATOGRAFIA',
       'Método..: IMUNOFLUORESCÊNCIA INDIRETA',
       'Método..: ENZIMOIMUNOENSAIO', 'Resultado conferido e liberado.',
       'Método..: QUIMIOLUMINESCÊNCIA', 'Detectado', 'Material:SORO',
       'Método..: PCR EM TEMPO REAL QUALITATIVO',
       'Swab nasofaringe/orofaringe', 'Positivo', 'Líquor',
       'Método..: RT-PCR QUALITATIVO EM TEMPO REAL',
       'Método..: PCR EM TEMPO REAL', 'Não Detectado',
       'Swab de Nasofaringe/Orofaringe', 'Material:BRAÇO', 'NASOFARINGE',
       'Análise sob restrição: Amostra em temperatura inadequada.',
       'nasofaringe', 'SWAB OROFARINGE/NASOFARINGE',
       'Swab Nasofaringe/Orofaringe', 'Material:soro',
       'SWAB NASO FARINGE', 'SORO',
       'e falso-negativo, que é uma característica do método.',
       'Nota: Este exame pode apresentar, embora raramente, resultado falso

In [61]:
##

In [62]:
## mostrar valores únicos da coluna Procedimento e Resultado, SOMENTE com números no valor, somar ocorreˆncias
dbmol_pat[dbmol_pat['Resultado'].str.contains('\d')]['Resultado'].value_counts()

0.2               6716
0.1               5755
0.3               4269
0.4               2643
Inferior a 0.1    1976
                  ... 
87.7                 1
307.8                1
48.7                 1
31.3                 1
7.41                 1
Name: Resultado, Length: 1920, dtype: int64

In [65]:
## fazer um dataframe  apontando os valores da coluna Procedimento e o min e max da coluna Resultado, sem repetir valores, sem campos com letras

dbmol_BI2itps = dbmol_pat[dbmol_pat['HasNumber']].groupby('Procedimento')['Resultado'].agg(['min', 'max'])
## se tiver a palavra inferior, considerar como valor min, se tiver a palavra superior considerar o valor max
dbmol_BI2itps['min'] = dbmol_BI2itps['min'].str.extract(r'(\d+\.\d+|\d+)')
# dbmol_BI2itps['Resultado'] = dbmol_BI2itps['min'] + ' - ' + dbmol_BI2itps['max']
dbmol_BI2itps


,min,max
Procedimento,,
ANTICORPOS ANTI CHIKUNGUNYA IGG E IGM,0.1,Inferior a 0.1
CHIKUNGUNYA VÍRUS IGG,0.1,Inferior a 0.1
CHIKUNGUNYA VÍRUS IGM,0.1,Inferior a 0.1
CORONAVÍRUS 2019 - SARS-COV-2 IGG QUANTITATIVO,10.0,Superior a 1800.0
CORONAVÍRUS 2019 ANTICORPOS IGA (COVID19),0.26,9.98
DENGUE - ANTICORPOS IGG,0.1,Inferior a 5.0
DENGUE - ANTICORPOS IGM,0.1,Inferior a 0.1
DETECÇÃO E TIPAGEM DO VÍRUS DA DENGUE,1,DENGUE SOROTIPO 1
ESTUDO SOROLÓGICO - VÍRUS FEBRE AMARELA IGG E IGM,1,Reagente 1/400


In [64]:
## agroupar por Procedimento e Resultado, baseado nos unique values da coluna Min e Max
dbmol_BI2 = dbmol_pat.groupby('Procedimento')['Resultado'].agg(['unique', 'min', 'max'])
dbmol_BI2

,unique,min,max
Procedimento,,,
ANTICORPOS ANTI CHIKUNGUNYA IGG E IGM,"[0.1, 0.2, Método..: ENZIMAIMUNOENSAIO, 0.4, R...",0.1,"Resultado conferido, repetido e liberado."
CHIKUNGUNYA VÍRUS IGG,"[Método..: ENZIMAIMUNOENSAIO, 0.3, 1.3, Inferi...",0.1,Resultado conferido e liberado.
CHIKUNGUNYA VÍRUS IGM,"[Resultado conferido e liberado., Método..: EN...",0.1,Resultado conferido e liberado.
CORONAVÍRUS 2019 - SARS-COV-2 IGG QUANTITATIVO,"[440.9, Método..: QUIMIOLUMINESCÊNCIA, 687.5, ...",10.0,Superior a 1800.0
CORONAVÍRUS 2019 ANTICORPOS IGA (COVID19),"[Resultado conferido e liberado., 8.56, Método...",0.26,Resultado conferido e liberado.
DENGUE - ANTICORPOS IGG,"[0.1, Método..: ENZIMAIMUNOENSAIO, 1.1, 0.2, 2...",0.1,"Resultado conferido, repetido e liberado."
DENGUE - ANTICORPOS IGM,"[0.2, Método..: ENZIMAIMUNOENSAIO, 0.1, 0.3, 0...",0.1,"Resultado conferido, repetido e liberado."
DENGUE NS1,"[Negativo, Método..: IMUNOCROMATOGRAFIA, Posit...",Meio de coleta: Soro com gel separador,Positivo
DETECÇÃO E TIPAGEM DO VÍRUS DA DENGUE,"[e falso-negativo, que é uma característica do...",DENGUE SOROTIPO 1,"e falso-negativo, que é uma característica do ..."
